In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cvxpy as cp

sns.set()

## First, Load the Data

In [172]:
path_to_prices = r'C:\Users\Rafay\Documents\thesis\ActualWork\Data\results\prices.csv'
path_to_factors = r'C:\Users\Rafay\Documents\thesis\ActualWork\Data\results\3factors.csv'
prices = pd.read_csv(path_to_prices)
factors = pd.read_csv(path_to_factors)
assets_list = list(prices['symbol'].unique())
prices.head()

,Unnamed: 0,symbol,date,open,high,low,close,volume,adjclose,splits,dividends
0,0,AMZN,2006-01-03,2.3735,2.3925,2.3125,2.3790,151644000,2.3790,0.0,0.0
1,1,AMZN,2006-01-04,2.3745,2.3865,2.3345,2.3625,148818000,2.3625,0.0,0.0
2,2,AMZN,2006-01-05,2.3580,2.4100,2.3555,2.3825,108344000,2.3825,0.0,0.0
3,3,AMZN,2006-01-06,2.3985,2.4290,2.3660,2.3935,123058000,2.3935,0.0,0.0
4,4,AMZN,2006-01-09,2.3275,2.3550,2.3200,2.3540,178862000,2.3540,0.0,0.0


In [6]:
pivot_prices = np.round(pd.pivot_table(prices, values='close', 
                                index='date', 
                                columns='symbol', 
                                aggfunc=np.mean),2)
pivot_prices = pivot_prices.reset_index()
pivot_prices['date'] = pd.to_datetime(pivot_prices['date'])
factors['date'] = pd.to_datetime(factors['Date'], format="%Y%m%d")

pivot_prices.head()

symbol,date,A,AAPL,ABB,ABT,ACN,ADBE,ADI,ADP,ADSK,...,WELL,WFC,WM,WMB,WPC,WTRG,WY,XEL,XOM,YUM
0,2006-01-03,23.96,2.67,10.19,18.98,29.30,38.52,36.70,36.74,42.75,...,34.55,31.90,30.67,19.48,25.74,22.05,66.64,18.57,58.47,16.80
1,2006-01-04,24.03,2.68,10.48,19.00,29.46,38.42,37.04,36.86,42.52,...,34.92,31.53,30.27,19.61,26.14,22.15,68.25,18.66,58.57,16.95
2,2006-01-05,24.66,2.66,10.55,19.25,29.68,38.07,38.22,36.55,41.87,...,35.64,31.50,30.39,19.40,27.20,22.31,67.65,18.65,58.28,17.67
3,2006-01-06,24.79,2.72,10.64,19.62,31.15,39.00,38.37,36.87,44.29,...,35.79,31.68,30.54,19.48,26.62,22.44,67.75,18.72,59.43,17.62
4,2006-01-09,24.71,2.72,10.95,20.35,31.06,38.38,38.91,36.77,43.64,...,36.30,31.67,31.04,19.79,27.00,22.50,67.59,18.67,59.40,17.76


In [7]:
pivot_prices = pivot_prices.set_index('date')
returns = pivot_prices.pct_change()
pivot_prices = pivot_prices.reset_index()
returns = returns.reset_index()
returns = returns.merge(factors, on='date', how='left')
returns = returns.drop(['Date'], axis=1)
returns = returns.dropna()
returns.describe()

,A,AAPL,ABB,ABT,ACN,ADBE,ADI,ADP,ADSK,AEE,...,WPC,WTRG,WY,XEL,XOM,YUM,Mkt-RF,SMB,HML,RF
count,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,...,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000,3774.000000
mean,0.000617,0.001254,0.000483,0.000558,0.000711,0.000904,0.000556,0.000519,0.000814,0.000212,...,0.000423,0.000321,0.000162,0.000415,0.000049,0.000648,0.030220,-0.005257,-0.011089,0.004483
std,0.019759,0.020795,0.020749,0.013860,0.016458,0.021275,0.019470,0.014627,0.024392,0.014647,...,0.017828,0.015733,0.024531,0.012631,0.016720,0.017376,1.051192,0.425688,0.414513,0.007001
min,-0.110117,-0.179039,-0.176218,-0.097857,-0.134543,-0.190346,-0.166149,-0.152526,-0.158599,-0.173750,...,-0.177856,-0.129066,-0.618934,-0.126918,-0.139525,-0.188261,-9.610000,-5.390000,-3.110000,0.000000
25%,-0.008193,-0.008296,-0.008518,-0.006159,-0.006860,-0.008508,-0.008628,-0.005821,-0.011073,-0.006411,...,-0.006701,-0.007085,-0.009605,-0.005654,-0.007461,-0.007099,-0.380000,-0.210000,-0.210000,0.000000
50%,0.000953,0.000809,0.000895,0.000395,0.000966,0.001173,0.000623,0.000764,0.001043,0.000582,...,0.000487,0.000767,0.000593,0.000557,0.000000,0.000632,0.080000,0.010000,-0.020000,0.000000
75%,0.010247,0.011663,0.010005,0.007652,0.008430,0.010885,0.009898,0.007501,0.012529,0.007422,...,0.007561,0.007984,0.010542,0.006963,0.007675,0.008395,0.510000,0.210000,0.180000,0.010000
max,0.138682,0.138728,0.219495,0.109360,0.163668,0.177193,0.156463,0.118576,0.161985,0.178014,...,0.206257,0.196295,0.253155,0.112577,0.171905,0.232484,9.200000,2.360000,4.160000,0.020000


In [139]:
'''
Inputs: AssetReturns: Pandas DataFrame, each date is a row, one column for each asset
        FactorReturn: Pandas DataFrame, each date is a row, one column for each factor
        Technique: Method through which parameters will be estimated. Default: OLS
Outputs: mu: numpy array, key: Symbol. value: return estimate
         Q: nxn Asset Covariance Matrix (n: # of assets)
'''
def GetParameterEstimates(AssetReturns, FactorReturns, technique='OLS', log=True):
    # Only have OLS implemented so far
    if technique!='OLS':
        return [], []
    
    AssetReturns_np = AssetReturns.to_numpy()
    FactorReturns_np = FactorReturns.to_numpy()
    T,n = AssetReturns_np.shape
    _, p = FactorReturns.shape

    # Get Data Matrix - Factors
    X = np.zeros((T, p+1))
    X[:,:-1] = np.ones((T,1)) # Add ones to first row
    X[:,1:] = FactorReturns

    # Get regression coefficients for Assets
    # B = (X^TX)^(-1)X^Ty
    B = np.matmul(np.linalg.inv((np.matmul(np.transpose(X), X))), (np.matmul(np.transpose(X), AssetReturns_np)))

    # Get alpha and betas
    a = np.transpose(B[0,:])
    V = B[1:(p+1),:]

    # Residual Variance to get D
    ep = AssetReturns_np - np.matmul(X, B)
    sigma_ep = 1/(T-p-1) * np.sum(np.square(ep), axis=0)
    D = np.diag(sigma_ep)

    # Get Factor Estimated Return and Covariance Matrix
    f_bar = np.transpose(np.mean(FactorReturns_np, axis=0))
    F = np.cov(FactorReturns_np, rowvar=False)

    # Get mu
    mu = a + np.matmul(np.transpose(V), f_bar)

    # Get Q
    Q = np.matmul(np.matmul(np.transpose(V), F), V) + D
    
    if log:
        print("Shape of X: {}".format(X.shape))
        print("Shape of B: {}".format(B.shape))
        print("Shape of X*B: {}".format(np.matmul(X, B).shape))
        print("Shape of ep: {}".format(ep.shape))
        print("Shape of sigma_ep: {}".format(sigma_ep.shape))
        print("Shape of D: {}".format(sigma_ep.shape))
        print("Shape of Q: {}".format(Q.shape))
    
    return mu, Q


In [297]:
'''
Mean Variance Optimizer
Inputs: mu: numpy array, key: Symbol. value: return estimate
        Q: nxn Asset Covariance Matrix (n: # of assets)
Outputs: x: optimal allocations
'''

def MVO(mu,Q):
    
    # # of Assets
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)
    
    # Target Return for Constraint
    targetRet = np.mean(mu)
    
    constraints = [
        cp.sum(w) == 1, # Sum to 1
        mu.T @ w >= targetRet, # Target Return Constraint
        w>=0 # Disallow Short Sales
    ]

    # Objective Function
    risk = cp.quad_form(w, Q)

    prob = cp.Problem(cp.Minimize(risk), constraints=constraints)
    prob.solve()
    return w.value

'''
Risk Parity Optimizer
Inputs: mu: numpy array, key: Symbol. value: return estimate
        Q: nxn Asset Covariance Matrix (n: # of assets)
Outputs: x: optimal allocations
'''

def RP(mu,Q):
    
    # # of Assets
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)
    k = cp.Parameter()
    k.value = 2
      
    constraints = [
        w>=0 # Disallow Short Sales
    ]

    # Objective Function
    risk = cp.quad_form(w, Q)
    log_term = 0
    for i in range(n):
        log_term += cp.log(w[i])
    
    prob = cp.Problem(cp.Minimize(risk-(k*log_term)), constraints=constraints)
    prob.solve()
    return w.value



In [186]:
from datetime import datetime

def generate_date_list(data, start, end):
    start = datetime.fromisoformat(start)
    end = datetime.fromisoformat(end)

    # Train model from start_date to date
    mask = (data['date'] >= start) & (data['date'] <= end)
    data = data.loc[mask]
    return data.date.apply(lambda x: x.date()).unique().tolist()

In [288]:
def GetOptimalAllocation(mu, Q, technique=Optimizers.MVO):
    if technique == Optimizers.MVO:
        return MVO(mu,Q)
    if technique == Optimizers.RP:
        return RP(mu, Q)
    


In [278]:
holdings.tail()

,date,AMZN,HD,TM,MCD,NKE,LOW,SBUX,BKNG,TJX,...,ELP,CNP,BEP,CMS,AES,ATO,EVRG,LNT,WTRG,NI
246,2007-12-24,5.072953e-18,1.024154e-17,8.588222e-18,3.554671e-19,2.723189e-18,3.000301e-18,2.042286e-18,2.198148e-18,4.273029e-18,...,2.161910e-18,1.767227e-18,2.499855e-18,2.931365e-18,2.408269e-18,2.535364e-18,4.470124e-18,7.631428e-19,4.696119e-18,2.602013e-18
247,2007-12-26,3.273294e-18,9.019320e-18,9.190859e-18,4.461072e-19,2.956803e-18,2.298686e-18,2.073505e-18,2.467251e-18,3.852620e-18,...,2.951956e-18,3.250221e-18,2.384592e-18,3.240882e-18,4.038084e-18,2.690180e-18,5.237468e-18,6.975752e-19,4.161298e-18,2.945628e-18
248,2007-12-27,-4.273800e-18,8.736341e-18,3.280560e-18,4.463958e-19,4.386801e-18,-5.624506e-18,-1.797377e-18,-2.130647e-18,6.157132e-18,...,5.389854e-18,9.453689e-18,3.544024e-18,8.901569e-19,3.762836e-18,-8.721354e-19,5.009717e-18,-1.210742e-18,4.576795e-18,1.341756e-18
249,2007-12-28,-5.359711e-18,6.147883e-18,2.854312e-17,7.467458e-19,1.193863e-17,-4.996546e-18,1.266276e-18,5.059692e-18,3.944323e-18,...,1.670074e-17,3.085986e-17,1.133172e-17,4.905389e-18,1.686306e-17,6.891261e-18,2.438442e-17,-3.634220e-18,9.239691e-18,7.057133e-18
250,2007-12-31,-9.864277e-18,4.124536e-18,3.278748e-17,-1.276623e-18,1.199927e-17,-7.808735e-18,-5.370226e-20,4.745230e-18,3.045732e-18,...,1.827662e-17,3.597936e-17,1.265716e-17,4.937481e-18,1.872950e-17,6.446140e-18,2.797817e-17,-6.101689e-18,9.250642e-18,7.716535e-18


In [296]:
from tqdm import tqdm
date = '2017-06-12'
assets_list_cleaned = [x for x in assets_list if str(x) != 'nan']
holdings = pd.DataFrame(columns=['date']+assets_list_cleaned)
portVal = pd.DataFrame(columns=['date', 'Wealth'])
factors = ['Mkt-RF', 'SMB', 'HML']
InitialValue = 1000000 # $1,000,000
lookback = 30 # Number of days preceeding current date to train

dates = generate_date_list(returns, start='2007-01-01', end='2007-12-31')
first = True
for date in tqdm(dates):
    # Get Asset Prices for Today
    currentPrices = (pivot_prices[pivot_prices['date']==str(date)]
        .drop('date',axis=1)
        .values
        .flatten())
    
    # Update Portfolio Value
    if first:
        portVal.loc[len(portVal)] = [date] + [InitialValue]
        CurrentPortfolioValue = InitialValue
        first = False
    else:     
        CurrentPortfolioValue = np.dot(currentPrices,noShares)
        portVal.loc[len(portVal)] = [date] + [CurrentPortfolioValue]
        
    
    # We don't want the current date information, hence the lack of equality
    # Get last 30
    date = str(date)
    
    returns_lastn = returns[(returns['date'] < date)].tail(lookback)
    factor_returns = returns_lastn[factors]
    asset_returns = returns_lastn.drop(factors + ['date', 'RF'], axis=1)

    mu, Q = GetParameterEstimates(asset_returns, factor_returns, log=False)
    x = GetOptimalAllocation(mu, Q, Optimizers.RP)

    # Update Holdings
    holdings.loc[len(holdings)] = [date] + list(x)

    # Update shares held
    # 50% of 100k = 50k. If price is 100 we have 50,000/100=50 shares
    noShares = np.divide(x*CurrentPortfolioValue, currentPrices)

  0%|          | 0/251 [00:00<?, ?it/s]


NotImplementedError: Strict inequalities are not allowed.

In [199]:
def calculateSharpe(holdings):
    pass

def plotWealth(holdings):
    print(holdings.head())

def plotHoldings(holdings):
    pass


In [285]:
TODO:
1. Write Risk Parity solver in CVXOPT
2. Write the new Dist. Robust RP in CVXOPT
3. Figure out how to use cvxpylayers
4. Make wealth, SR and holdings graphs code

C:\Users\Rafay\AppData\Local\Temp\ipykernel_19260\3636925383.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  [x for x in list(round(holdings.sum(axis=1),3)) if x!=1.0]


[]

In [287]:
from enum import Enum
class Optimizers(Enum):
    MVO = 1
    RP = 2

In [143]:
Optimizers.MVO

<Optimizers.MVO: 1>